Load required libraries

In [1]:
%%capture
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers

import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from google.colab import userdata
from google.colab import drive

In [3]:
# Retrieve Fireworks AI API key
dotenv.load_dotenv()
fireworks.client.api_key = userdata.get('FW_API_KEY')

# Mount Google Drive with the RFP database
drive.mount('/content/drive')

!ls "/content/drive/My Drive/Colab Notebooks"

A function to get completions from the fireworks LLM

In [6]:
def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v3p1-8b-instruct"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

Test connectivity to the mistral LLM

In [7]:
mistral_llm = "mixtral-8x7b-instruct"

get_completion("[INST]Tell me 2 jokes[/INST]", model=mistral_llm)

" Sure, here are two jokes for you:\n\n1. Why don't scientists trust atoms?\n\nBecause they make up everything!\n\n1. Why did the scarecrow win an award?\n\nBecause he was outstanding"

Usecase: Read a database with RFP Questions and Answers to augment the text generation using the LLM. See: retrieval augmented generation.


In [68]:
# load dataset from google drive and convert to dataframe
# dataset contains column names
rfp_file = pd.read_csv('/content/drive/My Drive/Colab Notebooks/rfp_data.csv', header=0, delimiter=";")

# remove rows with empty titles or descriptions
rfp_qa = rfp_file.dropna(subset=["Question", "Answer"])
rfp_qa.head()

,Question,Answer
0,How is a systematic investment process develop...,The IQS team is committed to adding value for ...
1,How is a systematic process tested? Is there a...,Managing risk is an integral part of Invesco’s...
2,What controls does Invesco have for initial im...,"Before execution, the portfolio managers revie..."
3,Regular review schedules and processes?,"In terms of model and process improvements, th..."
4,Other activities performed to ensure models ar...,In addition to the regular review by the Resea...


In [69]:
# convert dataframe to list of dicts with Question and Answer columns only
rfp_qa_dict = rfp_qa.to_dict(orient="records")
rfp_qa_dict[0]

{'Question': 'How is a systematic investment process developed at Invesco?',
 'Answer': 'The IQS team is committed to adding value for clients through the systematic application of factor investing. \nIQS is convinced that certain factors such as Value, Momentum and Quality explain wide parts of both returns and risks in equity markets. However, while individual factors may deliver better returns than traditional market capitalization weighted benchmarks in the long run, short term drawdowns of single factors may be severe and consistently successful factor timing has proven to be challenging if not impossible. Hence, the team has very successfully been implementing broadly diversified multi-factor strategies for 40 years, seeking to capture factor premiums irrespective of the prevailing market environment and timing considerations.\nThe first step of the investment process is the definition of the investment universe, for this strategy the global market. In addition to meeting minimum

Install Chroma DB as a vector database to store word embeddings.

In [58]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        batch_embeddings = embedding_model.encode(input)
        return batch_embeddings.tolist()

embed_fn = MyEmbeddingFunction()

# Initialize the chromadb directory, and client.
client = chromadb.PersistentClient(path="./chromadb")

# create collection
collection = client.get_or_create_collection(
    name=f"rfp-materials-2024"
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [59]:
# Generate embeddings, and index titles in batches
batch_size = 50

# loop through batches and generated + store embeddings
for i in tqdm(range(0, len(rfp_qa_dict), batch_size)):

    i_end = min(i + batch_size, len(rfp_qa_dict))
    batch = rfp_qa_dict[i : i + batch_size]

    # Replace empty strings (shouldn't happen)
    batch_titles = [str(paper["Question"]) if str(paper["Question"]) != "" else "No Question" for paper in batch]
    batch_answers = [str(paper["Answer"]) if str(paper["Answer"]) != "" else "No Answer" for paper in batch]
    batch_ids = [str(sum(ord(c) + random.randint(1, 10000) for c in paper["Question"])) for paper in batch]
    #batch_metadata = [dict(url=paper["PaperURL"],
    #                       abstract=paper['Abstract'])
    #                       for paper in batch]

    # generate embeddings
    batch_embeddings = embedding_model.encode(batch_titles)

    # upsert to chromadb
    collection.upsert(
        ids=batch_ids,
    #    metadatas=batch_metadata,
    #    documents=batch_titles,
        documents=batch_answers,
        embeddings=batch_embeddings.tolist(),
    )

  0%|          | 0/3 [00:00<?, ?it/s]

Testing the text retriever, this will retrieve output based on the data found in the vector database.

In [71]:
collection = client.get_or_create_collection(
    name=f"rfp-materials-2024",
    embedding_function=embed_fn
)

retriever_results = collection.query(
    query_texts=["What are ESG factors?"],
    n_results=2,
)

print(retriever_results["documents"])

[['In addition to the standard ESG integration, IQS can implement individual ESG requirements in numerous respects:\n\n-\tPositive criteria: By applying positive criteria, companies are identified that display excellence in sustainable management and sustainable products or processes. They fulfil ecological and social requirements particularly well, ranging from climate efficiency and low water consumption to labour safety and satisfaction.\n-\tExclusion and negative criteria: By applying these criteria, companies, sectors or countries can be excluded from the investment universe which fail to fulfil certain ESG criteria or that violate international norms and standards according to the definitions of the International Labour Organisation (ILO), the OECD or the United Nations. \n\nThe investment team is able to define a set of ESG criteria in close co-operation with the client. Exclusion criteria and negative criteria can be used to eliminate companies that fail to meet certain ESG cri

In [67]:
# user query
user_query = "How is ESG integrated?"

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=3,
)

# concatenate titles into a single string
answers = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to generate RFP_TEXT by summarizing the RFP_ANSWERS.

You should mimic a style similar to the one in the QUESTION.

QUESTION: {user_query}

RFP_ANSWERS: {answers}

RFP_TEXT:

[/INST]
'''

model_response = get_completion(prompt_template, model=mistral_llm, max_tokens=500)

# Print the suggestions.
print("Your query")
print(">>> ", user_query)
print("***********************")

print("Model generated answer:")
print()
print(model_response)
print("***********************")

print("Reference answers from RFP database:")
print()
print(answers)
print("***********************")

Your query
>>>  How is ESG integrated?
***********************
Model generated answer:

IQS incorporates a comprehensive and integrated ESG (Environmental, Social, and Governance) investment process in managing client portfolios. This approach is based on Invesco's longstanding experience in customized ESG solutions, active engagement with companies, and the Invesco proxy voting approach. IQS has enhanced its risk management by introducing a dedicated ESG exposure control for all portfolios and an adverse ESG momentum measure to restrict companies experiencing severe downgrades to their ESG scores.

The adverse ESG momentum measure targets companies that have delivered highly disappointing news, causing the market to reassess their core processes and values. IQS has found evidence that these companies continue to underperform after downgrade events, and expects this trend to have a greater impact going forward due to the increased importance of ESG considerations. IQS treats such stock